配对交易是一种市场中性策略，它利用两种资产之间价格变动的相关性。本文的目的是研究使用可口可乐 （KO） 和百事可乐 （PEP） 股票的配对交易策略。可口可乐和百事可乐都在软饮料行业开展业务，尽管它们的财务基本面存在差异，包括与百事可乐的较大销售额相比，可口可乐的净收入更高，而且当新法规等行业范围的事件影响它们时，它们的股价往往朝着同一个方向移动。

本文将使用基于 Python 的方法来实施货币对交易策略，分析 KO 和 PEP 的价格走势，以便在它们的价格偏离其历史关系时识别有利可图的交易机会。通过将统计套利与传统的 50/50 买入并持有方法进行比较，我们深入了解了与统计套利相关的潜在好处和挑战。

In [6]:
import io 
import re 
import requests 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
plt.style.use("seaborn-v0_8") 

def getdata(tickers,start,end,frequency):
    OHLC = {} 
    cookie = ''
    crumb = '' 
    res = requests.get('https://finance.yahoo.com/quote/SPY/history')
    cookie = res.cookies['B']
    pattern = re.compile('.*"CrumbStore":\{"crumb":"(?P<crumb>[^"]+)"\}')
    for line in res.text.splitlines():
        m = pattern.match(line)
        if m is not None:
            crumb = m.groupdict()['crumb']
    for ticker in tickers:
        url_str = "https://query1.finance.yahoo.com/v7/finance/download/%s"
        url_str += "?period1=%s&period2=%s&interval=%s&events=history&crumb=%s"
        url = url_str % (ticker, start, end, frequency, crumb)
        res = requests.get(url, cookies={'B': cookie}).text
        OHLC[ticker] = pd.read_csv(io.StringIO(res), index_col=0,
                                   error_bad_lines=False).replace('null', np.nan).dropna()
        OHLC[ticker].index = pd.to_datetime(OHLC[ticker].index)
        OHLC[ticker] = OHLC[ticker].apply(pd.to_numeric)
    return OHLC
tickers = ["PEP","KO"] 
data = None 
while data is None:
    try:
        data = getdata(tickers,'946685000','1687427200','1d') 
    except:
        pass 
KO = data['KO'] 
PEP = data['PEP']

主函数 getdata 是一个函数，它接受股票代码列表、开始和结束日期以及数据可用性的频率。在脚本执行期间，它会创建一个字典，在其中存储每个代码的开盘价-最高价-最低价-收盘价 （OHLC） 数据。为了发出 API 请求，该函数首先请求 Yahoo Finance 的历史页面以获取 cookie 和 crumb。

该函数中有一个函数，该函数构造一个 URL，可以从中下载每个代码的历史数据，向 URL 发送 GET 请求，并将响应读入 pandas DataFrame。在此过程中，它会将所有 null 值替换为 NaN 并删除任何缺失的条目以清理数据。日期索引将转换为日期时间格式，并且所有值都更改为数字类型。

在代码中，初始化了一组 tickers，例如 PEP 和 KO，并且重复请求数据，直到成功获取数据。如果返回数据字典，则它会从中提取 KO 和 PEP 的 DataFrames，从而轻松访问和分析返回的 DataFrame 中的股票数据。

In [ ]:
pairs = pd.DataFrame({'TPEP':PEP['Close'].shift(1)/PEP['Close'].shift(2)-1,
                      'TKO':KO["Close"].shift(1)/KO['Close'].shift(2) - 1}) 
pairs['Target'] = pairs.min(axis=1) 
 # Signal that triggers the purchase of the asset
pairs['Correlation'] = ((PEP['Close'].shift(1)/PEP['Close'].shift(20)-1).rolling(window=9)
                        .corr((KO['Close'].shift(1)/KO['Close'].shift(20)-1)))

Signal = pairs['Correlation'] < 0.9

# We're holding positions that weren't profitable yesterday
HoldingYesterdayPosition = ((pairs['Target'].shift(1).isin(pairs['TPEP']) &
                             (PEP['Close'].shift(1)/PEP['Open'].shift(1)-1 < 0)) |
                            (pairs['Target'].shift(1).isin(pairs['TKO']) &
                             (KO['Close'].shift(1)/KO['Open'].shift(1)-1 < 0))) # if tc, add here

# Since we aren't using leverage, we can't enter on a new position if
# we entered on a position yesterday (and if it wasn't profitable)
NoMoney = Signal.shift(1) & HoldingYesterdayPosition
pairs['PEP'] = np.where(NoMoney,
                        np.nan,
                        np.where(PEP['Close']/PEP['Open']-1 < 0,
                                 PEP['Close'].shift(-1)/PEP['Open']-1,
                                 PEP['Close']/PEP['Open']-1)) 
pairs['KO'] = np.where(NoMoney,
                       np.nan,
                       np.where(KO['Close']/KO['Open']-1 < 0,
                                KO['Close'].shift(-1)/KO['Open']-1,
                                KO['Close']/KO['Open']-1))

pairs['Returns'] = np.where(Signal,
                            np.where(pairs['Target'].isin(pairs['TPEP']),
                                      pairs['PEP'],
                                      pairs['KO']),
                             np.nan) # if tc, add here

pairs['CumulativeReturn'] = pairs['Returns'].dropna().cumsum()

In [ ]:
print(plt.style.available)

通过使用 pandas 进行数据操作，此代码用于分析和交易两只股票，作为使用 pandas 进行数据分析的统计套利策略的一部分。此函数中有一个公式，通过比较前两天的收盘价来计算股票 PEP 和 KO 的每日回报。然后，这些返回值将存储在名为 pairs 的 DataFrame 中。在确定前一天回报率最低的股票后，Target （目标） 列用于记录哪只股票的回报率最低。

然后使用 9 天的滚动窗口来计算股票回报之间的相关性，以尝试评估它们最近的行为。当两个变量的相关性低于阈值 0.9 时，将生成购买信号。HoldingYesterdayPosition 变量提供有关昨天是否持有任何无利可图的头寸的信息。NoMoney 变量用于在存在购买信号但已经存在无利可图的持仓时防止进行新输入。

在此之后，代码根据每只股票的 NoMoney 条件确定每只股票将有多少回报。如果没有可用资金，将向 NaN 分配退货;否则，将检查今天的回报，如果今天的回报为负数，则使用明天的回报。使用 Target 股票作为计算回报的基础，计算总回报，并且可以在 Returns （回报） 列中看到。

同样重要的是要注意，该代码计算已执行交易随时间推移的累积回报，在 CumulativeReturn 列中提供利润或损失的运行总和，以便评估交易策略随时间推移的表现。

In [ ]:
ReturnPEP = PEP['Close']/PEP['Open']-1 
BuyHoldPEP = PEP['Adj Close']/float(PEP['Adj Close'][:1])-1 
# Coca Cola returns
ReturnKO = KO['Close']/KO['Open']-1
BuyHoldKO = KO['Adj Close']/float(KO['Adj Close'][:1])-1

# Benchmark
ReturnBoth = (ReturnPEP+ReturnKO)/2
BuyHoldBoth = ((BuyHoldPEP+BuyHoldKO)/2).fillna(method='ffill')

下面提供了一个计算指定时间段内百事可乐股票和可口可乐股票回报率的代码。首先，它通过将当前收盘价与前一天的开盘价进行比较来确定每只股票的每日回报。然后，结果表示为两个价格之间的百分比变化。作为衡量股票是否有可能实现买入并持有回报的指标，使用第一天调整后的收盘价与当前调整后的收盘价之间的百分比变化来计算买入并持有回报。最后但并非最不重要的一点是，该代码计算了两只股票的每日回报率的平均值以创建基准回报率，并计算了买入并持有回报率的平均值，并进行了前瞻性填充，以代替任何缺失的数据点，以确保完整的回报时间序列。

In [ ]:
returns = pairs['Returns'].dropna() 
cumulret = pairs['CumulativeReturn'].dropna() 

fig,ax = plt.subplots(figsize=(16,6)) 
hist1,bins1 = np.histogram(ReturnBoth.dropna(),bins=50) 
width = 0.7 * (bins1[1] - bins1[0]) 
center = (bins1[:-1] + bins1[1:]) / 2 
ax.bar(center,hist1,align='center',width=width,label='50/50 Returns') 
hist2,bins2 = np.histogram(returns,bins=50) 
ax.bar(center,hist2,align='center',width=width,label='Pairs Trading') 
plt.legend()
plt.show()

print('=====Strategy Returns=====')
print('Mean return =',round((returns.mean())*100,2),"%")
print('Standard deviaton =',round((returns.std())*100,2),"%")
print("==========================")
print('Worst return =',round((min(returns))*100,2),"%")
print('Best return =',round((max(returns))*100,2),"%")
print("=========================")
print('Lower quantile =',round((returns.quantile(q=0.25))*100,2),"%")
print('Median return =',round((returns.quantile(q=0.5))*100,2),"%")
print('Upper quantile =',round((returns.quantile(q=0.75))*100,2),"%")

此代码分析了与 50/50 回报设置相比，货币对交易策略的预期回报。此函数从名为 pair 的 DataFrame 中提取 Returns 和 CumulativeReturn 列，并确保删除任何缺失值，以确保结果尽可能清晰。

它创建了一个直方图，显示 50 个区间内的返回分布，并调整了条形宽度以更准确地表示分布。第一个直方图显示 50/50 的回报，而第二个直方图显示交易回报的货币对，两者都绘制在同一轴上，以便于比较。

查看可视化后，代码会计算货币对交易回报的各种统计数据，例如平均值、标准差、最差和最佳回报，以及基于可视化的下四分位数、中位数和上四分位数。这些值四舍五入到小数点后两位，并以百分比形式显示，从而可以清楚地了解策略在一段时间内的表现。

In [ ]:
executionrate = len(returns) / len(ReturnBoth) 

maxdd = round(max(np.maximum.accumulate(cumulret) - cumulret)*100,2) 
mask = returns < 0 
diffs = np.diff(mask.astype(int)) 
start_mask = np.append(True,diffs==1) 
mask1 = mask & ~(start_mask & np.append(diffs==-1,True)) 
id = (start_mask & mask1).cumsum() 
out = np.bincount(id[mask1]-1,returns[mask1]) 
badd = round(max(-out)*100,2) 
spositive = returns[returns > 0] 
snegative = -returns[returns < 0] 
winrate = round((len(spositive)/(len(spositive)+len(snegative)))*100,2)

beta = round(returns.corr(ReturnBoth),2)

sharpe = round((float(cumulret[-1:]))/cumulret.std(),2)

tret = round((float(cumulret[-1:]))*100,2)

下面显示了一个代码片段，该代码片段计算与投资回报相关的各种指标，侧重于风险和绩效。执行率是通过将 returns 数组的长度与另一个名为 ReturnBoth 的数组进行比较来确定的，该数组根据 returns 数组和 ReturnBoth 数组的长度之间的比率，提供对已执行交易数量的洞察。然后，它通过将累积回报与最大值进行比较，以找到它与峰值之间的最大差异，从而计算出最大回撤，该回撤代表累积回报从峰值到谷底的最大跌幅，以百分比表示。

由于编写了代码来识别负回报期间，因此创建了一个掩码来跟踪这些事件，并应用了一个筛选条件来筛选掉正期间之后的短暂负期间。Badd 是负回报导致的回撤的最大百分比，是通过计算负回报的贡献来计算的。使用正回报和负回报作为衡量投资策略成功的基础，胜率计算为正回报相对于策略中回报总数的百分比。

它还通过 beta 和 ReturnBoth 之间的相关性计算，以反映与 ReturnBoth 相比回报的波动性。为了计算夏普比率，我们必须考虑累积回报和这些回报的标准差，这表明一个波动率单位的平均回报率高于无风险利率。最后，tret 计算为最后的累积回报率，它代表投资在一段时间内实现的回报金额。

In [ ]:
plt.figure(figsize=(16,6))
plt.plot(BuyHoldBoth*100, label='Buy & Hold 50/50')
plt.plot(cumulret*100, label='Pairs Trading', color='coral')
plt.xlabel('Time')
plt.ylabel('Returns (in %)')
plt.margins(x=0.005,y=0.02)
plt.axhline(y=0, xmin=0, xmax=1, linestyle='--', color='k')
plt.legend()
plt.show()

print("Cumulative Return = ",tret,"%")
print("=========================")
print("Execution Rate = ",round(executionrate*100,2),"%")
print("Win Rate = ",winrate,"%")
print("=========================")
print("Maximum Loss = ",maxdd,"%")
print("Maximum Consecutive Loss = ",badd,"%")
print("=========================")
print("Beta = ",beta)
print("Sharpe = ",sharpe)
# Return ("alpha") decay

通过使用Matplotlib库创建了一个图表，以比较两种投资策略：在此代码中买入并持有50/50和配对交易。设置一个具有定义大小的数字，并将两种策略的累积回报绘制在图上，回报按比例缩放以表示每种策略的百分比回报。此图表的 x 轴标记为 Time，y 轴标记为 Returned in percent，并调整了边距以获得更好的视觉间距。作为参考点，在 y=0 级别绘制带有虚线的箭头。

在最终可视化中，图例区分了两条绘制的线，并且可以查看最终的可视化。除了将多个性能指标打印到控制台之外，该代码还打印有关系统性能的多个指标，包括累积回报、执行率、胜率、最大损失、最大连续损失、beta 和夏普比率。作为此摘要的结果，我们能够评估与这些策略相关的有效性和风险指标，同时指出自 2011 年以来回报的下降可能归因于过度拟合和未能将回报再投资。

In [ ]:
BuyHoldBothYTD = (((PEP['Adj Close'][-252:]/float(PEP['Adj Close'][-252])-1)+(KO['Adj Close'][-252:]/float(KO['Adj Close'][-252])-1))/2).fillna(method='ffill')
StrategyYTD = returns[-92:].cumsum()

plt.figure(figsize=(16,6))
plt.plot(BuyHoldBothYTD*100, label='Buy & Hold 50/50')
plt.plot(StrategyYTD*100, label='Pairs Trading', color='coral')
plt.xlabel('Time')
plt.ylabel('Returns (in %)')
plt.margins(x=0.005,y=0.02)
plt.axhline(y=0, xmin=0, xmax=1, linestyle='--', color='k')
plt.legend()
plt.show()

print('Buy & Hold 50/50 YTD Performance (at 1 July 2020) =',round(float(BuyHoldBothYTD[-1:]*100),1),'%')
print('Strategy YTD Performance =',round(float(StrategyYTD[-1:]*100),1),'%')